In [1]:
import numpy as np
from pathlib import Path, PosixPath, WindowsPath
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import xarray as xr

Path to dataset

In [2]:
# path = PosixPath('/media/davidhaasnoot/files/large_data_sets/CAMELS')
path = WindowsPath('D:/large_data_sets/CAMELS')

# load characteristics 

In [3]:
indicator_files = list(path.glob('*.txt'))

In [4]:
df_clim_indicators = pd.read_csv(indicator_files[0],delimiter=";")
df_geol_indicators = pd.read_csv(indicator_files[1],delimiter=";")
df_hydro_indicators = pd.read_csv(indicator_files[2],delimiter=";")
df_catchment_names = pd.read_csv(indicator_files[3],delimiter=";")
df_soil_indicators = pd.read_csv(indicator_files[4],delimiter=";")
df_topo_indicators = pd.read_csv(indicator_files[5],delimiter=";")
df_vege_indicators = pd.read_csv(indicator_files[6],delimiter=";")
df_vege_indicators['dom_land_cover'] = df_vege_indicators['dom_land_cover'].apply(lambda x: x.strip())

In [5]:
df_char_lst = [df_geol_indicators,
                df_clim_indicators,
                df_hydro_indicators,
                df_catchment_names,
                df_soil_indicators,
                df_vege_indicators, 
                df_topo_indicators]
for index, df in enumerate(df_char_lst):
    df_char_lst[index] = df.set_index('gauge_id')

In [6]:
characteristics_all = pd.concat(df_char_lst,axis=1)

## Forcing path

In [7]:
source_lst = ['daymet','maurer','nldas']
source = source_lst[0]
forcing_sub_path_1 = f'basin_timeseries_v1p2_metForcing_obsFlow'
forcing_sub_path_2 = f'basin_dataset_public_v1p2'
forcing_sub_path_3 = f'basin_mean_forcing'

In [8]:
forcing_path = path / forcing_sub_path_1 / forcing_sub_path_2 / forcing_sub_path_3 / source

In [9]:
sub_dirs_forcing = list(forcing_path.glob("*"))

In [10]:
basin_mean_forcing_files = list(sub_dirs_forcing[0].glob("*.txt"))

In [11]:
basin_mean_forcing_file = basin_mean_forcing_files[0]
basin_id = basin_mean_forcing_file.name[:8]
folder_id = basin_mean_forcing_file.parent.name

In [12]:
catchment_char = characteristics_all.loc[int(basin_id)].copy()

In [13]:
len(list(characteristics_all.index))

671

## retrieve alpha

retrieve alpha: not always 1.26

In [14]:
model_ouput_path = path / f'basin_timeseries_v1p2_modelOutput_{source}' / f'model_output_{source}' / 'model_output' /'flow_timeseries'/ f'{source}'

In [15]:
model_ouput_path

WindowsPath('D:/large_data_sets/CAMELS/basin_timeseries_v1p2_modelOutput_daymet/model_output_daymet/model_output/flow_timeseries/daymet')

In [16]:
parameter_files_path = list((model_ouput_path / f'{folder_id}').glob(f'{basin_id}_*_model_parameters.txt'))

In [17]:
line_start = "pet_coef"
lst_alpha = []
for file in parameter_files_path:
    with open(file,'r') as lines:
        for line in lines:
            if line[:len(line_start)] == line_start:
                lst_alpha.append(float(line.strip().split("\t")[-1]))

In [18]:
alpha = np.array(lst_alpha).mean()

## add streamflow

In [19]:
streamflow_path = path / forcing_sub_path_1 / forcing_sub_path_2 / 'usgs_streamflow'
streamflow_file_path = streamflow_path / f'{folder_id}' / f'{basin_id}_streamflow_qc.txt' 

## attributes

The excel was slightly edited to be the same format as the text files

In [20]:
df_meta_data_labels = pd.read_excel(path / 'camels_attributes_v2.0_edited.xlsx')
df_meta_data_labels.head(3)

,Attribute,Description,Unit,Data source,References
0,gauge_id,catchment identifier (8-digit USGS hydrologic ...,-,N15 - USGS data,NaN
1,huc_02,region (2-digit USGS hydrologic unit code),-,N15 - USGS data,NaN
2,gauge_name,"gauge name, followed by the state",-,N15 - USGS data,NaN


In [21]:
units_char = dict(zip(df_meta_data_labels['Attribute'], df_meta_data_labels['Unit']))
description_char = dict(zip(df_meta_data_labels['Attribute'], df_meta_data_labels['Description']))
source_char = dict(zip(df_meta_data_labels['Attribute'], df_meta_data_labels['Data source']))

In [22]:
units_dict = {
        'dayl': 's',
        'pr': 'mm/d',
        'srad': 'W/m2',
        'swe': 'mm',
        'tasmax': 'degC',
        'tasmin': 'degC',
        'vp': 'Pa',
        'evspsblpot': 'mm/d',
        'tas': 'degC',
        'streamflow': 'mm/d'}

units_dict.update(units_char)

In [23]:
longname_dict = {
        'dayl': 'daylight',
        'pr': 'precipitation',
        'srad': 'mean radiation',
        'swe': ' snow water equivalent',
        'tasmax': 'temperature max',
        'tasmin': 'temperature min',
        'vp': 'vapourpressure',
        'evspsblpot': 'potential evaporation',
        'tas': 'temerature mean',
        'streamflow': 'observed streamflow'}
longname_dict.update(description_char)

In [24]:
data_source_dict = {
        'dayl': 'See data_source variable',
        'pr': 'See data_source variable',
        'srad': 'See data_source variable',
        'swe': 'See data_source variable',
        'tasmax': 'See data_source variable',
        'tasmin': 'See data_source variable',
        'vp': 'See data_source variable',
        'evspsblpot': 'Computed using ',
        'tas': 'See data_source variable',
        'streamflow': 'USGS'}
data_source_dict.update(source_char)

# Combine all 

In [25]:
from read_camels import from_camels_txt, crop_ds, calc_pet

In [26]:
start_date = "1980-01-01T00:00:00Z"
end_date = "2014-12-31T00:00:00Z"

In [28]:
ouput_path = path.cwd() / "Output" 
ouput_path.mkdir(exist_ok=True)
out_path = ouput_path / source
out_path.mkdir(exist_ok=True)

In [29]:
ds0 = from_camels_txt(basin_mean_forcing_file, basin_id, source, start_date, end_date, out_path, alpha, catchment_char, streamflow_file_path,units_dict, longname_dict, data_source_dict)

## test merge

In [30]:
source_lst = ['daymet','maurer','nldas']
source = source_lst[0]
# Source dependant
forcing_sub_path_1 = f'basin_timeseries_v1p2_metForcing_obsFlow'
forcing_sub_path_2 = f'basin_dataset_public_v1p2'
forcing_sub_path_3 = f'basin_mean_forcing'
forcing_path = path / forcing_sub_path_1 / forcing_sub_path_2 / forcing_sub_path_3 / source
sub_dirs_forcing = list(forcing_path.glob("*"))
model_ouput_path = path / f'basin_timeseries_v1p2_modelOutput_{source}' / f'model_output_{source}' / 'model_output' /'flow_timeseries'/ f'{source}'
streamflow_path = path / forcing_sub_path_1 / forcing_sub_path_2 / 'usgs_streamflow'

In [31]:
# basin dependant
basin_mean_forcing_file = basin_mean_forcing_files[1]
basin_id = basin_mean_forcing_file.name[:8]
folder_id = basin_mean_forcing_file.parent.name
catchment_char = characteristics_all.loc[int(basin_id)].copy()
parameter_files_path = list((model_ouput_path / f'{folder_id}').glob(f'{basin_id}_*_model_parameters.txt'))
line_start = "pet_coef"
lst_alpha = []
for file in parameter_files_path:
    with open(file,'r') as lines:
        for line in lines:
            if line[:len(line_start)] == line_start:
                lst_alpha.append(float(line.strip().split("\t")[-1]))
alpha = np.array(lst_alpha).mean()

streamflow_file_path = streamflow_path / f'{folder_id}' / f'{basin_id}_streamflow_qc.txt' 

In [32]:
ds1 = from_camels_txt(basin_mean_forcing_file, basin_id, source, start_date, end_date, out_path, alpha, catchment_char, streamflow_file_path,units_dict, longname_dict, data_source_dict)

In [33]:
xr.merge([ds0,ds1])

<xarray.Dataset>
Dimensions:               (time: 12784, data_source: 1, basin_id: 2)
Coordinates:
  * time                  (time) datetime64[ns] 1980-01-01 ... 2014-12-31
  * data_source           (data_source) |S64 b'daymet'
  * basin_id              (basin_id) |S64 b'01013500' b'01022500'
Data variables: (12/69)
    dayl                  (basin_id, data_source, time) float64 3.017e+04 ......
    pr                    (basin_id, data_source, time) float64 0.0 0.0 ... 0.0
    srad                  (basin_id, data_source, time) float64 153.4 ... 187.3
    swe                   (basin_id, data_source, time) float64 0.0 0.0 ... 0.0
    tasmax                (basin_id, data_source, time) float64 -6.54 ... -7.25
    tasmin                (basin_id, data_source, time) float64 -16.3 ... -16.56
    ...                    ...
    gauge_lat             (basin_id) float64 47.24 44.61
    gauge_lon             (basin_id) float64 -68.58 -67.94
    elev_mean             (basin_id) float64 250.3 92.68
    slope_mean            (basin_id) float64 21.64 17.79
    area_gages2           (basin_id) float64 2.253e+03 573.6
    area_geospa_fabric    (basin_id) float64 2.304e+03 620.4
Attributes:
    title:            Basin mean forcing data
    history:          Converted to netCDF by David Haasnoot for eWatercycle u...
    data_source:      CAMELS was compiled by A. Newman et al. `A large-sample...
    url_source_data:  https://dx.doi.org/10.5065/D6MW2F4D
    alpha:            1.2606
    lat:              46.84
    elevation(m):     353.0
    area basin(m^2):  2260093113.0

# loop

not found 1150900, 6775500, 6846500 in catchment_characteristics

error in 02108000, 05120500, 07067000, 09492400 wrong header daymet

Make sure you fix the headers: the loop will fail

In [34]:
skip_ids = [2081113, 3448942, 9535100, 1150900, 6775500, 6846500]

In [35]:
df

,gauge_id,gauge_lat,gauge_lon,elev_mean,slope_mean,area_gages2,area_geospa_fabric
0,1013500,47.23739,-68.58264,250.31,21.64152,2252.70,2303.95
1,1022500,44.60797,-67.93524,92.68,17.79072,573.60,620.38
2,1030500,45.50097,-68.30596,143.80,12.79195,3676.17,3676.09
3,1031500,45.17501,-69.31470,247.80,29.56035,769.05,766.53
4,1047000,44.86920,-69.95510,310.38,49.92122,909.10,904.94
...,...,...,...,...,...,...,...
666,14309500,42.80400,-123.61091,709.83,110.42527,224.92,226.31
667,14316700,43.34984,-122.72894,952.26,119.08920,587.90,588.01
668,14325000,42.89150,-124.07065,656.53,124.96889,443.07,444.92
669,14362250,42.15401,-123.07532,875.67,109.93127,41.42,43.88


In [36]:
source_lst = ['daymet','maurer','nldas']
for source in source_lst:
    # Source dependant
    out_path = path.cwd() / "Output" / source
    out_path.mkdir(exist_ok=True)
    forcing_sub_path_1 = f'basin_timeseries_v1p2_metForcing_obsFlow'
    forcing_sub_path_2 = f'basin_dataset_public_v1p2'
    forcing_sub_path_3 = f'basin_mean_forcing'
    forcing_path = path / forcing_sub_path_1 / forcing_sub_path_2 / forcing_sub_path_3 / source
    sub_dirs_forcing = list(forcing_path.glob("*"))
    model_ouput_path = path / f'basin_timeseries_v1p2_modelOutput_{source}' / f'model_output_{source}' / 'model_output' /'flow_timeseries'/ f'{source}'
    streamflow_path = path / forcing_sub_path_1 / forcing_sub_path_2 / 'usgs_streamflow'
    
    for sub_dirs in sub_dirs_forcing:
        basin_mean_forcing_files = list(sub_dirs.glob("*.txt"))
        for basin_mean_forcing_file in basin_mean_forcing_files:
            basin_id = basin_mean_forcing_file.name[:8]
            if int(basin_id) in skip_ids:
                pass 
            else:
                folder_id = basin_mean_forcing_file.parent.name
                catchment_char = characteristics_all.loc[int(basin_id)].copy()
                parameter_files_path = list((model_ouput_path / f'{folder_id}').glob(f'{basin_id}_*_model_parameters.txt'))
                
                if source == "daymet":
                    line_start = "pet_coef"
                elif source != 'daymet':
                    line_start = "PT_COEF"
                lst_alpha = []    
                for file in parameter_files_path:
                    with open(file,'r') as lines:
                        for line in lines:
                            if line[:len(line_start)] == line_start:
                                lst_alpha.append(float(line.strip().split("\t")[-1]))
                alpha = np.array(lst_alpha).mean()
                
                streamflow_file_path = streamflow_path / f'{folder_id}' / f'{basin_id}_streamflow_qc.txt' 
                ds = from_camels_txt(basin_mean_forcing_file, 
                                     basin_id, 
                                     source, 
                                     start_date, 
                                     end_date, 
                                     out_path, 
                                     alpha, 
                                     catchment_char, 
                                     streamflow_file_path,
                                     units_dict,
                                     longname_dict,
                                     data_source_dict)
                ds.close()

In [3]:
files_daymet = list((Path.cwd() / "Output" / "daymet").glob("*nc"))
files_maurer = list((Path.cwd() / "Output" / "maurer").glob("*nc"))
files_nldas = list((Path.cwd() / "Output" / "nldas").glob("*nc"))

In [38]:
file_lst = []
names = files_daymet# + files_maurer + files_nldas
for file in names:
    basin_id = file.name[19:27]
    ds = xr.open_dataset(file, chunks={'time': 1000, 'basin_id': 100,'data_source':1 })
    ds = ds.expand_dims(dim={'basin_id':np.array([(str('camels_'+ basin_id)).encode()],dtype=np.dtype('|S64'))})
    file_lst.append(ds)

In [39]:
ds = xr.concat(file_lst,dim=('basin_id'))

In [40]:
time_vars = list(ds.data_vars)[:10]

In [41]:
# from https://gist.github.com/BSchilperoort/256751fe2ea060c50b103f72026590a2
compression = {'zlib': True,'complevel': 8}
encoding = {var: compression for var in time_vars}  # don't encode string-based vars!
#     output_file = path_output / f"{dataset}.nc"
#     print(f"Exporting {output_file}!")
#     ds.to_netcdf(output_file, encoding=encoding)

In [42]:
ds.to_netcdf("combined_daymet.nc", encoding=encoding)

In [43]:
ds.close()

In [5]:
file_lst = []
names = files_maurer # + files_nldas + files_daymet
for file in names:
    basin_id = file.name[19:27]
    ds = xr.open_dataset(file, chunks={'time': 6000, 'basin_id': 100,'data_source':1 })
    ds = ds.expand_dims(dim={'basin_id':np.array([(str('camels_'+ basin_id)).encode()],dtype=np.dtype('|S64'))})
    file_lst.append(ds)

In [6]:
ds = xr.concat(file_lst,dim=('basin_id'))

In [7]:
time_vars = list(ds.data_vars)[:10]
compression = {'zlib': True,'complevel': 8}
encoding = {var: compression for var in time_vars} 

In [8]:
ds.to_netcdf("combined_maurer.nc", encoding=encoding)

In [9]:
ds.close()

In [10]:
file_lst = []
names =  files_nldas # + files_daymet + files_maurer
for file in names:
    basin_id = file.name[19:27]
    ds = xr.open_dataset(file, chunks={'time': 1000, 'basin_id': 10,'data_source':1 })
    ds = ds.expand_dims(dim={'basin_id':np.array([(str('camels_'+ basin_id)).encode()],dtype=np.dtype('|S64'))})
    file_lst.append(ds)

In [11]:
ds = xr.concat(file_lst,dim=('basin_id'))

In [12]:
time_vars = list(ds.data_vars)[:10]
compression = {'zlib': True,'complevel': 8}
encoding = {var: compression for var in time_vars} 

In [13]:
ds.to_netcdf("combined_nldas.nc", encoding=encoding)

## combine the three

In [14]:
files = ["combined_nldas.nc","combined_maurer.nc","combined_daymet.nc"]
file_lst = []
for file in files:
    ds = xr.open_dataset(file, chunks={'time': 1000, 'basin_id': 100,'data_source':1 })
    file_lst.append(ds)

C:\Users\david\anaconda3\envs\uni\Lib\site-packages\xarray\core\dataset.py:275: UserWarning: The specified chunks separate the stored chunks along dimension "basin_id" starting at index 100. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(
C:\Users\david\anaconda3\envs\uni\Lib\site-packages\xarray\core\dataset.py:275: UserWarning: The specified chunks separate the stored chunks along dimension "time" starting at index 1000. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(
C:\Users\david\anaconda3\envs\uni\Lib\site-packages\xarray\core\dataset.py:275: UserWarning: The specified chunks separate the stored chunks along dimension "basin_id" starting at index 100. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(
C:\Users\david\anaconda3\envs\uni\Lib\site-packages\xarray\core\dataset.py:275: UserWarning: The specified chunks separate the stored chunks along dime

In [15]:
file_lst[0]

<xarray.Dataset>
Dimensions:               (basin_id: 671, time: 12784, data_source: 1)
Coordinates:
  * basin_id              (basin_id) |S15 b'camels_01013500' ... b'camels_144...
  * time                  (time) datetime64[ns] 1980-01-01 ... 2014-12-31
  * data_source           (data_source) |S64 b'nldas'
Data variables: (12/69)
    dayl                  (basin_id, data_source, time) float64 dask.array<chunksize=(100, 1, 1000), meta=np.ndarray>
    pr                    (basin_id, data_source, time) float64 dask.array<chunksize=(100, 1, 1000), meta=np.ndarray>
    srad                  (basin_id, data_source, time) float64 dask.array<chunksize=(100, 1, 1000), meta=np.ndarray>
    swe                   (basin_id, data_source, time) float64 dask.array<chunksize=(100, 1, 1000), meta=np.ndarray>
    tasmax                (basin_id, data_source, time) float64 dask.array<chunksize=(100, 1, 1000), meta=np.ndarray>
    tasmin                (basin_id, data_source, time) float64 dask.array<chunksize=(100, 1, 1000), meta=np.ndarray>
    ...                    ...
    gauge_lat             (basin_id) float64 dask.array<chunksize=(100,), meta=np.ndarray>
    gauge_lon             (basin_id) float64 dask.array<chunksize=(100,), meta=np.ndarray>
    elev_mean             (basin_id) float64 dask.array<chunksize=(100,), meta=np.ndarray>
    slope_mean            (basin_id) float64 dask.array<chunksize=(100,), meta=np.ndarray>
    area_gages2           (basin_id) float64 dask.array<chunksize=(100,), meta=np.ndarray>
    area_geospa_fabric    (basin_id) float64 dask.array<chunksize=(100,), meta=np.ndarray>
Attributes:
    title:            Basin mean forcing data
    history:          Converted to netCDF by David Haasnoot for eWatercycle u...
    data_source:      CAMELS was compiled by A. Newman et al. `A large-sample...
    url_source_data:  https://dx.doi.org/10.5065/D6MW2F4D
    alpha:            1.2620999999999998
    lat:              46.84
    elevation(m):     353.0
    area basin(m^2):  2260093113.0

In [16]:
ds = xr.concat(file_lst,dim=('data_source'))

In [17]:
ds

<xarray.Dataset>
Dimensions:               (basin_id: 671, time: 12784, data_source: 3)
Coordinates:
  * basin_id              (basin_id) |S15 b'camels_01013500' ... b'camels_144...
  * time                  (time) datetime64[ns] 1980-01-01 ... 2014-12-31
  * data_source           (data_source) |S64 b'nldas' b'maurer' b'daymet'
Data variables: (12/69)
    dayl                  (basin_id, data_source, time) float64 dask.array<chunksize=(100, 1, 1000), meta=np.ndarray>
    pr                    (basin_id, data_source, time) float64 dask.array<chunksize=(100, 1, 1000), meta=np.ndarray>
    srad                  (basin_id, data_source, time) float64 dask.array<chunksize=(100, 1, 1000), meta=np.ndarray>
    swe                   (basin_id, data_source, time) float64 dask.array<chunksize=(100, 1, 1000), meta=np.ndarray>
    tasmax                (basin_id, data_source, time) float64 dask.array<chunksize=(100, 1, 1000), meta=np.ndarray>
    tasmin                (basin_id, data_source, time) float64 dask.array<chunksize=(100, 1, 1000), meta=np.ndarray>
    ...                    ...
    gauge_lat             (data_source, basin_id) float64 dask.array<chunksize=(1, 100), meta=np.ndarray>
    gauge_lon             (data_source, basin_id) float64 dask.array<chunksize=(1, 100), meta=np.ndarray>
    elev_mean             (data_source, basin_id) float64 dask.array<chunksize=(1, 100), meta=np.ndarray>
    slope_mean            (data_source, basin_id) float64 dask.array<chunksize=(1, 100), meta=np.ndarray>
    area_gages2           (data_source, basin_id) float64 dask.array<chunksize=(1, 100), meta=np.ndarray>
    area_geospa_fabric    (data_source, basin_id) float64 dask.array<chunksize=(1, 100), meta=np.ndarray>
Attributes:
    title:            Basin mean forcing data
    history:          Converted to netCDF by David Haasnoot for eWatercycle u...
    data_source:      CAMELS was compiled by A. Newman et al. `A large-sample...
    url_source_data:  https://dx.doi.org/10.5065/D6MW2F4D
    alpha:            1.2620999999999998
    lat:              46.84
    elevation(m):     353.0
    area basin(m^2):  2260093113.0

I removed excess columns:

In [18]:
new_atrrs = {'title': 'Basin mean forcing data',
 'history': 'Converted to netCDF by David Haasnoot for eWatercycle using CAMELS dataset',
 'data_source': 'CAMELS was compiled by A. Newman et al. `A large-sample watershed-scale hydrometeorological dataset for the contiguous USA`',
 'url_source_data': 'https://dx.doi.org/10.5065/D6MW2F4D'}

In [19]:
ds.attrs = new_atrrs

In [20]:
time_vars = list(ds.data_vars)[:10]
compression = {'zlib': True,'complevel': 8}
encoding = {var: compression for var in time_vars} 

In [21]:
ds

<xarray.Dataset>
Dimensions:               (basin_id: 671, time: 12784, data_source: 3)
Coordinates:
  * basin_id              (basin_id) |S15 b'camels_01013500' ... b'camels_144...
  * time                  (time) datetime64[ns] 1980-01-01 ... 2014-12-31
  * data_source           (data_source) |S64 b'nldas' b'maurer' b'daymet'
Data variables: (12/69)
    dayl                  (basin_id, data_source, time) float64 dask.array<chunksize=(100, 1, 1000), meta=np.ndarray>
    pr                    (basin_id, data_source, time) float64 dask.array<chunksize=(100, 1, 1000), meta=np.ndarray>
    srad                  (basin_id, data_source, time) float64 dask.array<chunksize=(100, 1, 1000), meta=np.ndarray>
    swe                   (basin_id, data_source, time) float64 dask.array<chunksize=(100, 1, 1000), meta=np.ndarray>
    tasmax                (basin_id, data_source, time) float64 dask.array<chunksize=(100, 1, 1000), meta=np.ndarray>
    tasmin                (basin_id, data_source, time) float64 dask.array<chunksize=(100, 1, 1000), meta=np.ndarray>
    ...                    ...
    gauge_lat             (data_source, basin_id) float64 dask.array<chunksize=(1, 100), meta=np.ndarray>
    gauge_lon             (data_source, basin_id) float64 dask.array<chunksize=(1, 100), meta=np.ndarray>
    elev_mean             (data_source, basin_id) float64 dask.array<chunksize=(1, 100), meta=np.ndarray>
    slope_mean            (data_source, basin_id) float64 dask.array<chunksize=(1, 100), meta=np.ndarray>
    area_gages2           (data_source, basin_id) float64 dask.array<chunksize=(1, 100), meta=np.ndarray>
    area_geospa_fabric    (data_source, basin_id) float64 dask.array<chunksize=(1, 100), meta=np.ndarray>
Attributes:
    title:            Basin mean forcing data
    history:          Converted to netCDF by David Haasnoot for eWatercycle u...
    data_source:      CAMELS was compiled by A. Newman et al. `A large-sample...
    url_source_data:  https://dx.doi.org/10.5065/D6MW2F4D

In [22]:
ds.to_netcdf("combined_CAMELS.nc", encoding=encoding)

In [23]:
ds.close()

#  Combine with caravan

In [24]:
chunks = {'time': 1200, 'basin_id': 300,'data_source':1 }

In [25]:
ds_caravan = xr.open_dataset("camels_CARAVAN.nc", chunks=chunks)

C:\Users\david\anaconda3\envs\uni\Lib\site-packages\xarray\core\dataset.py:275: UserWarning: The specified chunks separate the stored chunks along dimension "basin_id" starting at index 300. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(
C:\Users\david\anaconda3\envs\uni\Lib\site-packages\xarray\core\dataset.py:275: UserWarning: The specified chunks separate the stored chunks along dimension "time" starting at index 1200. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(


In [26]:
ds_caravan

<xarray.Dataset>
Dimensions:                             (basin_id: 482, time: 14609)
Coordinates:
  * time                                (time) datetime64[ns] 1981-01-01 ... ...
  * basin_id                            (basin_id) <U15 'camels_01022500' ......
Data variables: (12/55)
    snow_depth_water_equivalent_mean    (basin_id, time) float32 dask.array<chunksize=(300, 1200), meta=np.ndarray>
    surface_net_solar_radiation_mean    (basin_id, time) float32 dask.array<chunksize=(300, 1200), meta=np.ndarray>
    surface_net_thermal_radiation_mean  (basin_id, time) float32 dask.array<chunksize=(300, 1200), meta=np.ndarray>
    surface_pressure_mean               (basin_id, time) float32 dask.array<chunksize=(300, 1200), meta=np.ndarray>
    temperature_2m_mean                 (basin_id, time) float32 dask.array<chunksize=(300, 1200), meta=np.ndarray>
    dewpoint_temperature_2m_mean        (basin_id, time) float32 dask.array<chunksize=(300, 1200), meta=np.ndarray>
    ...                                  ...
    moisture_index                      (basin_id) float64 dask.array<chunksize=(300,), meta=np.ndarray>
    seasonality                         (basin_id) float64 dask.array<chunksize=(300,), meta=np.ndarray>
    high_prec_freq                      (basin_id) float64 dask.array<chunksize=(300,), meta=np.ndarray>
    high_prec_dur                       (basin_id) float64 dask.array<chunksize=(300,), meta=np.ndarray>
    low_prec_freq                       (basin_id) float64 dask.array<chunksize=(300,), meta=np.ndarray>
    low_prec_dur                        (basin_id) float64 dask.array<chunksize=(300,), meta=np.ndarray>
Attributes:
    history:  Merged together from separate files; All forcing and state vari...

need to add data source: era5-land

In [27]:
ds_caravan = ds_caravan.expand_dims(dim={'data_source':np.array([(str('era5_land')).encode()],dtype=np.dtype('|S64'))})

In [28]:
ds_caravan

<xarray.Dataset>
Dimensions:                             (data_source: 1, basin_id: 482,
                                         time: 14609)
Coordinates:
  * data_source                         (data_source) object b'era5_land'
  * time                                (time) datetime64[ns] 1981-01-01 ... ...
  * basin_id                            (basin_id) <U15 'camels_01022500' ......
Data variables: (12/55)
    snow_depth_water_equivalent_mean    (data_source, basin_id, time) float32 dask.array<chunksize=(1, 300, 1200), meta=np.ndarray>
    surface_net_solar_radiation_mean    (data_source, basin_id, time) float32 dask.array<chunksize=(1, 300, 1200), meta=np.ndarray>
    surface_net_thermal_radiation_mean  (data_source, basin_id, time) float32 dask.array<chunksize=(1, 300, 1200), meta=np.ndarray>
    surface_pressure_mean               (data_source, basin_id, time) float32 dask.array<chunksize=(1, 300, 1200), meta=np.ndarray>
    temperature_2m_mean                 (data_source, basin_id, time) float32 dask.array<chunksize=(1, 300, 1200), meta=np.ndarray>
    dewpoint_temperature_2m_mean        (data_source, basin_id, time) float32 dask.array<chunksize=(1, 300, 1200), meta=np.ndarray>
    ...                                  ...
    moisture_index                      (data_source, basin_id) float64 dask.array<chunksize=(1, 300), meta=np.ndarray>
    seasonality                         (data_source, basin_id) float64 dask.array<chunksize=(1, 300), meta=np.ndarray>
    high_prec_freq                      (data_source, basin_id) float64 dask.array<chunksize=(1, 300), meta=np.ndarray>
    high_prec_dur                       (data_source, basin_id) float64 dask.array<chunksize=(1, 300), meta=np.ndarray>
    low_prec_freq                       (data_source, basin_id) float64 dask.array<chunksize=(1, 300), meta=np.ndarray>
    low_prec_dur                        (data_source, basin_id) float64 dask.array<chunksize=(1, 300), meta=np.ndarray>
Attributes:
    history:  Merged together from separate files; All forcing and state vari...

In [29]:
ds_camels = xr.open_dataset("combined_CAMELS.nc", chunks=chunks)

C:\Users\david\anaconda3\envs\uni\Lib\site-packages\xarray\core\dataset.py:275: UserWarning: The specified chunks separate the stored chunks along dimension "basin_id" starting at index 300. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(
C:\Users\david\anaconda3\envs\uni\Lib\site-packages\xarray\core\dataset.py:275: UserWarning: The specified chunks separate the stored chunks along dimension "time" starting at index 1200. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(


In [30]:
ds_camels

<xarray.Dataset>
Dimensions:               (basin_id: 671, time: 12784, data_source: 3)
Coordinates:
  * basin_id              (basin_id) |S15 b'camels_01013500' ... b'camels_144...
  * time                  (time) datetime64[ns] 1980-01-01 ... 2014-12-31
  * data_source           (data_source) |S64 b'nldas' b'maurer' b'daymet'
Data variables: (12/69)
    dayl                  (basin_id, data_source, time) float64 dask.array<chunksize=(300, 1, 1200), meta=np.ndarray>
    pr                    (basin_id, data_source, time) float64 dask.array<chunksize=(300, 1, 1200), meta=np.ndarray>
    srad                  (basin_id, data_source, time) float64 dask.array<chunksize=(300, 1, 1200), meta=np.ndarray>
    swe                   (basin_id, data_source, time) float64 dask.array<chunksize=(300, 1, 1200), meta=np.ndarray>
    tasmax                (basin_id, data_source, time) float64 dask.array<chunksize=(300, 1, 1200), meta=np.ndarray>
    tasmin                (basin_id, data_source, time) float64 dask.array<chunksize=(300, 1, 1200), meta=np.ndarray>
    ...                    ...
    gauge_lat             (data_source, basin_id) float64 dask.array<chunksize=(1, 300), meta=np.ndarray>
    gauge_lon             (data_source, basin_id) float64 dask.array<chunksize=(1, 300), meta=np.ndarray>
    elev_mean             (data_source, basin_id) float64 dask.array<chunksize=(1, 300), meta=np.ndarray>
    slope_mean            (data_source, basin_id) float64 dask.array<chunksize=(1, 300), meta=np.ndarray>
    area_gages2           (data_source, basin_id) float64 dask.array<chunksize=(1, 300), meta=np.ndarray>
    area_geospa_fabric    (data_source, basin_id) float64 dask.array<chunksize=(1, 300), meta=np.ndarray>
Attributes:
    title:            Basin mean forcing data
    history:          Converted to netCDF by David Haasnoot for eWatercycle u...
    data_source:      CAMELS was compiled by A. Newman et al. `A large-sample...
    url_source_data:  https://dx.doi.org/10.5065/D6MW2F4D

In [31]:
ds_out = xr.concat([ds_caravan, ds_camels], dim='data_source')

In [32]:
lst_float_vars = []
for var in ds_out.data_vars:
    if ds_out.data_vars[var].dtype == np.dtype(float) or ds_out.data_vars[var].dtype == np.dtype('float32'):
        lst_float_vars.append(var)


In [33]:
compression = {'zlib': True,'complevel': 8}
encoding = {var: compression for var in lst_float_vars} 

In [34]:
# ds_out.to_netcdf("camels_updated.nc", encoding=encoding)

In [35]:
# ds_out.close()

In [36]:
# ds_out = xr.open_dataset("camels_updated.nc", chunks=chunks)

In [37]:
ds_out.attrs

{'history': 'Merged together from separate files; All forcing and state variables are derived from ERA5-Land hourly by ECMWF. Streamflow data was taken from the CAMELS (US) dataset by Newman et al. (2014).'}

In [38]:
new_attrs  = {'title': 'Basin mean forcing data for ',
 'history': 'Converted to netCDF by David Haasnoot for eWatercycle using CAMELS and combined with merged Caravan dataset',
 'data_source_camels': 'CAMELS-USA was compiled by A. Newman et al. `A large-sample watershed-scale hydrometeorological dataset for the contiguous USA` using daymet, nldas and maurer',
 'data_source_caravan': 'The Caravan dataset by F. Kratzert et al `Caravan - A global community dataset for large-sample hydrology` uses era5 data',
 'url_source_data_camels': 'https://dx.doi.org/10.5065/D6MW2F4D',
 'url_source_data_caravan': 'https://doi.org/10.1038/s41597-023-01975-w',
             }

In [39]:
ds_out.attrs = new_attrs

In [40]:
ds_out

<xarray.Dataset>
Dimensions:                             (time: 14975, basin_id: 1153,
                                         data_source: 4)
Coordinates:
  * time                                (time) datetime64[ns] 1980-01-01 ... ...
  * basin_id                            (basin_id) <U15 'camels_01022500' ......
  * data_source                         (data_source) object b'era5_land' ......
Data variables: (12/115)
    snow_depth_water_equivalent_mean    (data_source, basin_id, time) float32 dask.array<chunksize=(4, 300, 1566), meta=np.ndarray>
    surface_net_solar_radiation_mean    (data_source, basin_id, time) float32 dask.array<chunksize=(4, 300, 1566), meta=np.ndarray>
    surface_net_thermal_radiation_mean  (data_source, basin_id, time) float32 dask.array<chunksize=(4, 300, 1566), meta=np.ndarray>
    surface_pressure_mean               (data_source, basin_id, time) float32 dask.array<chunksize=(4, 300, 1566), meta=np.ndarray>
    temperature_2m_mean                 (data_source, basin_id, time) float32 dask.array<chunksize=(4, 300, 1566), meta=np.ndarray>
    dewpoint_temperature_2m_mean        (data_source, basin_id, time) float32 dask.array<chunksize=(4, 300, 1566), meta=np.ndarray>
    ...                                  ...
    gauge_lat                           (data_source, basin_id) float64 dask.array<chunksize=(2, 782), meta=np.ndarray>
    gauge_lon                           (data_source, basin_id) float64 dask.array<chunksize=(2, 782), meta=np.ndarray>
    elev_mean                           (data_source, basin_id) float64 dask.array<chunksize=(2, 782), meta=np.ndarray>
    slope_mean                          (data_source, basin_id) float64 dask.array<chunksize=(2, 782), meta=np.ndarray>
    area_gages2                         (data_source, basin_id) float64 dask.array<chunksize=(2, 782), meta=np.ndarray>
    area_geospa_fabric                  (data_source, basin_id) float64 dask.array<chunksize=(2, 782), meta=np.ndarray>
Attributes:
    title:                    Basin mean forcing data for 
    history:                  Converted to netCDF by David Haasnoot for eWate...
    data_source_camels:       CAMELS-USA was compiled by A. Newman et al. `A ...
    data_source_caravan:      The Caravan dataset by F. Kratzert et al `Carav...
    url_source_data_camels:   https://dx.doi.org/10.5065/D6MW2F4D
    url_source_data_caravan:  https://doi.org/10.1038/s41597-023-01975-w

In [41]:
ds_out.to_netcdf("camels_updated_correct_attrs.nc", encoding=encoding)

In [42]:
ds_out.sel(data_source='era5_land'.encode()).isel(basin_id=0)

<xarray.Dataset>
Dimensions:                             (time: 14975)
Coordinates:
  * time                                (time) datetime64[ns] 1980-01-01 ... ...
    basin_id                            <U15 'camels_01022500'
    data_source                         |S9 b'era5_land'
Data variables: (12/115)
    snow_depth_water_equivalent_mean    (time) float32 dask.array<chunksize=(1566,), meta=np.ndarray>
    surface_net_solar_radiation_mean    (time) float32 dask.array<chunksize=(1566,), meta=np.ndarray>
    surface_net_thermal_radiation_mean  (time) float32 dask.array<chunksize=(1566,), meta=np.ndarray>
    surface_pressure_mean               (time) float32 dask.array<chunksize=(1566,), meta=np.ndarray>
    temperature_2m_mean                 (time) float32 dask.array<chunksize=(1566,), meta=np.ndarray>
    dewpoint_temperature_2m_mean        (time) float32 dask.array<chunksize=(1566,), meta=np.ndarray>
    ...                                  ...
    gauge_lat                           float64 dask.array<chunksize=(), meta=np.ndarray>
    gauge_lon                           float64 dask.array<chunksize=(), meta=np.ndarray>
    elev_mean                           float64 dask.array<chunksize=(), meta=np.ndarray>
    slope_mean                          float64 dask.array<chunksize=(), meta=np.ndarray>
    area_gages2                         float64 dask.array<chunksize=(), meta=np.ndarray>
    area_geospa_fabric                  float64 dask.array<chunksize=(), meta=np.ndarray>
Attributes:
    title:                    Basin mean forcing data for 
    history:                  Converted to netCDF by David Haasnoot for eWate...
    data_source_camels:       CAMELS-USA was compiled by A. Newman et al. `A ...
    data_source_caravan:      The Caravan dataset by F. Kratzert et al `Carav...
    url_source_data_camels:   https://dx.doi.org/10.5065/D6MW2F4D
    url_source_data_caravan:  https://doi.org/10.1038/s41597-023-01975-w